Perceber faiss, e esquema todo

In [ ]:
"""
def load_h5_data(source_dir):
    h5Files = sorted(glob.glob(source_dir + '**/*.h5', recursive=True))

    embs_count = 0
    embs_info = []
    embs = []
    music_names = []

    for i in range(len(h5Files)):
        with h5py.File(h5Files[i], "r") as f:
            #print(i)
            base_name = os.path.splitext(os.path.basename(h5Files[i]))[0]
            #primeiro objeto é o que contém os embeddings
            a_group_key = list(f.keys())[0]

            #Extração dos embs como um array
            ds_arr = f[a_group_key][()]  # returns as a numpy array
            #print(ds_arr.shape)
            embeddings = np.squeeze(ds_arr, axis=1)
            #print(embeddings.shape)
            embs.append(embeddings) #Guarda na lista os embs
            #arrayEmb = ds_arr

            embs_count += len(ds_arr) #conta quantos embs tem o vetor
            embs_info.append([i, base_name, embs_count]) #guarda numa lista o número de vetores até o momento.
            # embs_info = [indice, file_name, n_segs]

            music_names.extend([base_name] * len(ds_arr))
            
            f.close()
    return embs, embs_info, music_names

"""

In [51]:
import os
import sys
import time
import glob
import click
import curses
import numpy as np
import faiss
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
#from eval_RA.utils.get_index_faiss import get_index
from eval_RA.utils.print_table import PrintTable

Carrega os vetores memmap, é apenas a função

In [18]:
def load_memmap_data(source_dir,
                     fname,
                     append_extra_length=None,
                     shape_only=False,
                     display=True):
    path_shape = source_dir + fname + '_shape.npy'
    path_data = source_dir + fname + '.mm'
    data_shape = np.load(path_shape)
    if shape_only:
        return data_shape

    if append_extra_length:
        data_shape[0] += append_extra_length
        data = np.memmap(path_data, dtype='float32', mode='r+',
                         shape=(data_shape[0], data_shape[1]))
    else:
        data = np.memmap(path_data, dtype='float32', mode='r',
                         shape=(data_shape[0], data_shape[1]))
    if display:
        print(f'Load {data_shape[0]:,} items from \033[32m{path_data}\033[0m.')
    return data, data_shape

In [17]:
emb_dir = '/mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/'
emb_dummy_dir=None
index_type='ivfpq'
nogpu=True
max_train=1e7
test_ids='icassp'
test_seq_len='1 3 5'#9 11 19' #numero de segmentos para os segunintes segundos: 1, 2, 3, 5, 6, 10
k_probe=20
display_interval=5

In [19]:
test_seq_len = np.asarray(
    list(map(int, test_seq_len.split())))  # '1 3 5' --> [1, 3, 5]

# Load items from {query, db, dummy_db}
query, query_shape = load_memmap_data(emb_dir, 'query')
db, db_shape = load_memmap_data(emb_dir, 'db')

if emb_dummy_dir is None:
    emb_dummy_dir = emb_dir
dummy_db, dummy_db_shape = load_memmap_data(emb_dummy_dir, 'dummy_db')

Load 29,500 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/query.mm.
Load 29,500 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/db.mm.
Load 53,754,198 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/dummy_db.mm.


In [5]:
"""dummy_db = dummy_db[:10000,:]
dummy_db_shape = np.array(dummy_db.shape)
db = db[:500,:]
db_shape= np.array(db.shape)
query= query[:500,:]
query_shape= np.array(query.shape)"""

'dummy_db = dummy_db[:10000,:]\ndummy_db_shape = np.array(dummy_db.shape)\ndb = db[:500,:]\ndb_shape= np.array(db.shape)\nquery= query[:500,:]\nquery_shape= np.array(query.shape)'

In [20]:
dummy_db_shape, db_shape, query_shape

(array([53754198,      128]), array([29500,   128]), array([29500,   128]))

In [21]:
def get_index(index_type,
              train_data,
              train_data_shape,
              use_gpu=True,
              max_nitem_train=2e7):

    # Fingerprint dimension, d
    d = 128#train_data_shape[1]

    # Build a flat (CPU) index
    index = faiss.IndexFlatL2(d) #

    mode = index_type.lower()
    print(f'Creating index: \033[93m{mode}\033[0m')

    if mode == 'ivfpq':
        # Using IVF-PQ index
        code_sz = 64 # power of 2
        n_centroids = 256#
        nbits = 8  # nbits must be 8, 12 or 16, The dimension d should be a multiple of M.
        index = faiss.IndexIVFPQ(index, d, n_centroids, code_sz, nbits) #nbits = número de bits por índice


    # Train index
    # len(train_data) = 53754198
    # max_nitem_train = 10000000.0
    max_nitem_train = int(max_nitem_train)
    start_time = time.time()
    if len(train_data) > max_nitem_train:
        print('Training index using {:>3.2f} % of data...'.format(
            100. * max_nitem_train / len(train_data)))
        
        # shuffle and reduce training data
        sel_tr_idx = np.random.permutation(len(train_data))

        #print(len(sel_tr_idx))
        

        sel_tr_idx = sel_tr_idx[:max_nitem_train]
        index.train(train_data[sel_tr_idx,:])
    else:
        print('Training index...')
        index.train(train_data) # Actually do nothing for {'l2', 'hnsw'}
    print('Elapsed time: {:.2f} seconds.'.format(time.time() - start_time))

    # N probe
    index.nprobe = 40
    return index

In [22]:
index = get_index(index_type, dummy_db, dummy_db.shape, (not nogpu),
                    max_train)
#index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')
#index.add(db); print(f'{len(db)} items from reference DB')

Creating index: ivfpq
Training index using 18.60 % of data...


In [ ]:
%%time
index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')

53754198 items from dummy DB
CPU times: user 48min 40s, sys: 3.38 s, total: 48min 43s
Wall time: 7min 20s


In [ ]:
#index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')
#index.add(db); print(f'{len(db)} items from reference DB')
#%%time
index.add(db); print(f'{len(db)} items from reference DB')

29500 items from reference DB


In [14]:
print(f'Added total {index.ntotal} items to DB.') #53783698

Added total 53783698 items to DB.


In [15]:
db.shape

(29500, 128)

In [16]:
query, query.shape, db, db.shape

(memmap([[ 0.04492491,  0.1270966 ,  0.08505773, ..., -0.14611703,
           0.04584039, -0.2061284 ],
         [ 0.169376  ,  0.06089051, -0.02685473, ..., -0.04961549,
          -0.06049845, -0.15369163],
         [ 0.10719954,  0.08526957, -0.11996314, ..., -0.16281487,
          -0.10421381, -0.10944535],
         ...,
         [ 0.16346152,  0.00257511,  0.03175417, ..., -0.02812611,
          -0.00081418, -0.00859264],
         [ 0.08251635, -0.01852906, -0.05228599, ..., -0.01777812,
          -0.01846879,  0.03699062],
         [ 0.01534273,  0.00481873, -0.04748525, ...,  0.0211641 ,
          -0.07289249,  0.09382156]], dtype=float32),
 (29500, 128),
 memmap([[-0.02820984,  0.05458414,  0.13315764, ..., -0.07943324,
          -0.06755827, -0.1349114 ],
         [-0.03512029, -0.0023555 , -0.01484859, ...,  0.06893457,
           0.0056461 , -0.10987652],
         [-0.12248736,  0.00509285,  0.0186034 , ..., -0.03802261,
          -0.01838935, -0.17803867],
         ...,
    

In [17]:
dummy_db, dummy_db.shape

(memmap([[-0.02531523,  0.04019322, -0.024949  , ..., -0.04270871,
           0.04694553, -0.2953804 ],
         [ 0.04895402,  0.02706368,  0.09538799, ..., -0.00557855,
           0.06903314, -0.3006253 ],
         [ 0.04895402,  0.02706368,  0.09538799, ..., -0.00557855,
           0.06903314, -0.3006253 ],
         ...,
         [ 0.03045553,  0.03002142,  0.05494113, ..., -0.01226748,
           0.02941038, -0.2961614 ],
         [ 0.03061585,  0.02999131,  0.05510492, ..., -0.01257171,
           0.02929047, -0.2961139 ],
         [ 0.03061585,  0.02999131,  0.05510492, ..., -0.01257171,
           0.02929047, -0.2961139 ]], dtype=float32),
 (53754198, 128))

## Prepare fake_recon_index

In [18]:
del dummy_db

In [19]:
query_shape

array([29500,   128])

In [20]:
# append_extra_length = 29500
# emb_dummy_dir = '/mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/'

fake_recon_index, index_shape = load_memmap_data(emb_dummy_dir, 'dummy_db', append_extra_length=query_shape[0], display=False) # shape: 53783698, 128

In [330]:
#fake_recon_index = fake_recon_index[:10500,:] 

In [21]:
index_shape = np.array(fake_recon_index.shape)

In [22]:
index_shape, fake_recon_index
# 53783698, 128

(array([53783698,      128]),
 memmap([[-2.5315231e-02,  4.0193215e-02, -2.4949001e-02, ...,
          -4.2708706e-02,  4.6945531e-02, -2.9538041e-01],
         [ 4.8954025e-02,  2.7063677e-02,  9.5387988e-02, ...,
          -5.5785514e-03,  6.9033138e-02, -3.0062529e-01],
         [ 4.8954025e-02,  2.7063677e-02,  9.5387988e-02, ...,
          -5.5785514e-03,  6.9033138e-02, -3.0062529e-01],
         ...,
         [ 6.0930941e-02,  6.7983478e-02,  2.2735108e-02, ...,
          -7.5465396e-02,  8.7168738e-02,  5.0816007e-02],
         [ 1.4986711e-02,  2.4725229e-04,  7.2701007e-02, ...,
          -1.0112375e-01,  9.6761830e-02,  8.0341525e-02],
         [-3.0574361e-02,  2.1421332e-02,  1.4270875e-01, ...,
          -1.5414906e-01,  5.8181353e-02,  9.7020291e-02]], dtype=float32))

In [23]:
# vai adicionar aos extra 29500 espaços vazios que tem o db

# dummy_db_shape[0]:dummy_db_shape[0] + query_shape[0] =[ 53754198 : 53754198+29500 ] =[ 53754198 : 53783698 ]

fake_recon_index[dummy_db_shape[0]:dummy_db_shape[0] + query_shape[0], :] = db[:, :]

In [24]:
fake_recon_index.flush()

In [25]:
db[0, :]

memmap([-0.02820984,  0.05458414,  0.13315764, -0.07217907,  0.00880428,
        -0.02644426, -0.0620158 ,  0.03395626,  0.22050709, -0.07457374,
         0.00176313, -0.02056349, -0.11613505,  0.05840379,  0.06076813,
         0.02107625,  0.06576505,  0.06772836, -0.09941087,  0.01126463,
         0.03091909, -0.00330019, -0.01438594,  0.07806544,  0.11890554,
         0.07657426,  0.03755019,  0.05498022,  0.05501391,  0.04497967,
        -0.00479582, -0.00982922, -0.15555283, -0.01359908,  0.07182553,
         0.01893048, -0.06315958, -0.10929884, -0.04172297,  0.06851115,
         0.04001686,  0.01183444, -0.05304273,  0.06989591, -0.12449499,
        -0.02797864, -0.01638886,  0.0504494 , -0.05245946, -0.10353263,
        -0.18387204, -0.08876616,  0.166653  , -0.08358106,  0.08487547,
         0.13269134, -0.04197102,  0.20488472,  0.15165468,  0.0330515 ,
        -0.01974918, -0.10508668, -0.03005299,  0.08300256, -0.06361946,
         0.05721753,  0.13452126,  0.14779134,  0.1

In [26]:
fake_recon_index[ 53754198,:]

memmap([-0.02820984,  0.05458414,  0.13315764, -0.07217907,  0.00880428,
        -0.02644426, -0.0620158 ,  0.03395626,  0.22050709, -0.07457374,
         0.00176313, -0.02056349, -0.11613505,  0.05840379,  0.06076813,
         0.02107625,  0.06576505,  0.06772836, -0.09941087,  0.01126463,
         0.03091909, -0.00330019, -0.01438594,  0.07806544,  0.11890554,
         0.07657426,  0.03755019,  0.05498022,  0.05501391,  0.04497967,
        -0.00479582, -0.00982922, -0.15555283, -0.01359908,  0.07182553,
         0.01893048, -0.06315958, -0.10929884, -0.04172297,  0.06851115,
         0.04001686,  0.01183444, -0.05304273,  0.06989591, -0.12449499,
        -0.02797864, -0.01638886,  0.0504494 , -0.05245946, -0.10353263,
        -0.18387204, -0.08876616,  0.166653  , -0.08358106,  0.08487547,
         0.13269134, -0.04197102,  0.20488472,  0.15165468,  0.0330515 ,
        -0.01974918, -0.10508668, -0.03005299,  0.08300256, -0.06361946,
         0.05721753,  0.13452126,  0.14779134,  0.1

In [27]:
test_ids = np.load(glob.glob('./**/test_ids_icassp2021.npy', recursive=True)[0])

In [28]:
dummy_db_shape[0] 

53754198

In [29]:
test_ids = np.array([14655, 24, 121, 487, 112, 483])

In [30]:
test_ids

array([14655,    24,   121,   487,   112,   483])

In [33]:
test_ids = test_ids[:1]

In [34]:
n_test = len(test_ids)
gt_ids  = test_ids + dummy_db_shape[0]
top1_exact = np.zeros((n_test, len(test_seq_len))).astype(int) # (n_test, test_seg_len)
top1_near = np.zeros((n_test, len(test_seq_len))).astype(int)
top3_exact = np.zeros((n_test, len(test_seq_len))).astype(int)
top10_exact = np.zeros((n_test, len(test_seq_len))).astype(int)

In [35]:
gt_ids # 53768853 - 53754198 = 14655

array([53768853])

In [ ]:
for ti, test_id in enumerate(test_ids):
    gt_id = gt_ids[ti]
    for si, sl in enumerate(test_seq_len):
        assert test_id <= len(query)

In [36]:
ti = 0
test_id = test_ids[ti]
gt_id = gt_ids[ti]
si = 1
sl = test_seq_len[si]

In [37]:
ti, test_id, gt_id, sl

(0, 14655, 53768853, 3)

In [40]:
q = query[test_id:(test_id + sl), :] # shape(q) = (length, dim)

In [41]:
q

memmap([[ 0.15809532, -0.02599731,  0.01563343, -0.01498089, -0.08509405,
          0.00918309,  0.07498254,  0.14784455,  0.04171429,  0.00654791,
          0.15193549,  0.0434388 , -0.08942915,  0.20740546,  0.03593424,
         -0.06072055, -0.04684889, -0.09035619, -0.06467165, -0.00962286,
         -0.01251085, -0.08126785,  0.00619965, -0.04982934, -0.00722066,
         -0.0855777 ,  0.17162009, -0.07458217,  0.04839539,  0.00616283,
          0.02169314, -0.07166849,  0.14735067,  0.02083374,  0.1120945 ,
          0.07894135, -0.0452273 , -0.10079931, -0.08433416, -0.0737057 ,
         -0.12632951, -0.10347243, -0.06913505,  0.07233108, -0.09519916,
          0.08077845, -0.03659033,  0.01063322, -0.11243182,  0.12478732,
          0.00699937, -0.00058141,  0.0937693 ,  0.04863384, -0.00416125,
         -0.06676556,  0.02907668, -0.02814799,  0.02993005, -0.00958559,
         -0.08623658,  0.00639441, -0.06963412, -0.01728812, -0.18978092,
         -0.02648345, -0.15898466,  0.

In [42]:
#np.savetxt('query3vetores.csv', q, delimiter=',')

In [43]:
query.shape

(29500, 128)

In [44]:
q.shape

(3, 128)

In [45]:
query[test_id, :]

memmap([ 0.15809532, -0.02599731,  0.01563343, -0.01498089, -0.08509405,
         0.00918309,  0.07498254,  0.14784455,  0.04171429,  0.00654791,
         0.15193549,  0.0434388 , -0.08942915,  0.20740546,  0.03593424,
        -0.06072055, -0.04684889, -0.09035619, -0.06467165, -0.00962286,
        -0.01251085, -0.08126785,  0.00619965, -0.04982934, -0.00722066,
        -0.0855777 ,  0.17162009, -0.07458217,  0.04839539,  0.00616283,
         0.02169314, -0.07166849,  0.14735067,  0.02083374,  0.1120945 ,
         0.07894135, -0.0452273 , -0.10079931, -0.08433416, -0.0737057 ,
        -0.12632951, -0.10347243, -0.06913505,  0.07233108, -0.09519916,
         0.08077845, -0.03659033,  0.01063322, -0.11243182,  0.12478732,
         0.00699937, -0.00058141,  0.0937693 ,  0.04863384, -0.00416125,
        -0.06676556,  0.02907668, -0.02814799,  0.02993005, -0.00958559,
        -0.08623658,  0.00639441, -0.06963412, -0.01728812, -0.18978092,
        -0.02648345, -0.15898466,  0.00409927,  0.0

In [47]:
%%timeit
# segment-level top k search for each segment
_, I = index.search(q, k_probe) # _: distance, I: result IDs matrix

In [48]:
I

array([[53768853, 44703565, 53768854, 53768832, 32602405,  3046219,
        50090820, 44703566, 32602390, 30244778, 25233445, 50021449,
        53768875, 28396311, 46040053, 27606284, 48439861, 39399655,
        39485449, 44703570],
       [53768855, 53768876, 50867759, 23104134, 53768854, 53768856,
        53768877, 50865421, 23104135, 23104273, 18441413, 23104138,
        23104097, 19577881, 23104107, 39696799, 23180610, 19577880,
        23104139, 50865250],
       [53768855, 53768856, 53768835, 50865430, 30777645, 53768834,
        32228339, 53768876, 53768878, 32228337, 30803754, 30803668,
        53768877, 23180587, 50865313, 32228338, 32216725, 50701614,
        21509771, 21509707]])

I fez a procura dos 20 índices mais próximos para cada segmento. <br><br>
array([<br><br>[53761792, 53761793, 53761770, 53761781,  5362706,  7275719,<br>
        24526547,  5362584,  6420704,  7295985, 24526136,  3680268,<br>
         3679969,  7275692, 53761804, 38301458,  7295993, 24526237,<br>
         3679974, 24526761],<br><br>
       [ 7270958, 53761771, 33433913, 24526599, 23316112, 34480776,<br>
         7351273, 47324267, 21135530, 53761794, 30379957, 53761769,<br>
        37939031, 13406213, 24526509, 37751473, 21135531,  8731950,<br>
        13406120, 27355198],<br><br>
       [28438719, 21019091, 50131108, 52704559, 52704441, 31186157,<br>
        30396079, 12989699, 31739697, 47163137, 34751522, 30396077,<br>
        31739698, 44823085,  7646024, 20132263, 41764769, 27870591,<br>
        30396076, 47163136]<br><br>])

In [49]:
# offset compensation to get the start IDs of candidate sequences
for offset in range(len(I)):
    print(offset)
    I[offset, :] -= offset

0
1
2


In [50]:
I

array([[53768853, 44703565, 53768854, 53768832, 32602405,  3046219,
        50090820, 44703566, 32602390, 30244778, 25233445, 50021449,
        53768875, 28396311, 46040053, 27606284, 48439861, 39399655,
        39485449, 44703570],
       [53768854, 53768875, 50867758, 23104133, 53768853, 53768855,
        53768876, 50865420, 23104134, 23104272, 18441412, 23104137,
        23104096, 19577880, 23104106, 39696798, 23180609, 19577879,
        23104138, 50865249],
       [53768853, 53768854, 53768833, 50865428, 30777643, 53768832,
        32228337, 53768874, 53768876, 32228335, 30803752, 30803666,
        53768875, 23180585, 50865311, 32228336, 32216723, 50701612,
        21509769, 21509705]])

In [51]:
I[np.where(I >= 0)]

array([53768853, 44703565, 53768854, 53768832, 32602405,  3046219,
       50090820, 44703566, 32602390, 30244778, 25233445, 50021449,
       53768875, 28396311, 46040053, 27606284, 48439861, 39399655,
       39485449, 44703570, 53768854, 53768875, 50867758, 23104133,
       53768853, 53768855, 53768876, 50865420, 23104134, 23104272,
       18441412, 23104137, 23104096, 19577880, 23104106, 39696798,
       23180609, 19577879, 23104138, 50865249, 53768853, 53768854,
       53768833, 50865428, 30777643, 53768832, 32228337, 53768874,
       53768876, 32228335, 30803752, 30803666, 53768875, 23180585,
       50865311, 32228336, 32216723, 50701612, 21509769, 21509705])

In [52]:
# unique candidates
candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0

In [53]:
candidates

array([ 3046219, 18441412, 19577879, 19577880, 21509705, 21509769,
       23104096, 23104106, 23104133, 23104134, 23104137, 23104138,
       23104272, 23180585, 23180609, 25233445, 27606284, 28396311,
       30244778, 30777643, 30803666, 30803752, 32216723, 32228335,
       32228336, 32228337, 32602390, 32602405, 39399655, 39485449,
       39696798, 44703565, 44703566, 44703570, 46040053, 48439861,
       50021449, 50090820, 50701612, 50865249, 50865311, 50865420,
       50865428, 50867758, 53768832, 53768833, 53768853, 53768854,
       53768855, 53768874, 53768875, 53768876])

In [54]:
""" Sequence match score """
_scores = np.zeros(len(candidates))

In [55]:
_scores

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [56]:
ci = 0
cid = candidates[ci] 

In [57]:
cid

3046219

In [58]:
cid+sl#, ou seja, se sl=3, 242058+3=242061

3046222

In [59]:
q

memmap([[ 0.15809532, -0.02599731,  0.01563343, -0.01498089, -0.08509405,
          0.00918309,  0.07498254,  0.14784455,  0.04171429,  0.00654791,
          0.15193549,  0.0434388 , -0.08942915,  0.20740546,  0.03593424,
         -0.06072055, -0.04684889, -0.09035619, -0.06467165, -0.00962286,
         -0.01251085, -0.08126785,  0.00619965, -0.04982934, -0.00722066,
         -0.0855777 ,  0.17162009, -0.07458217,  0.04839539,  0.00616283,
          0.02169314, -0.07166849,  0.14735067,  0.02083374,  0.1120945 ,
          0.07894135, -0.0452273 , -0.10079931, -0.08433416, -0.0737057 ,
         -0.12632951, -0.10347243, -0.06913505,  0.07233108, -0.09519916,
          0.08077845, -0.03659033,  0.01063322, -0.11243182,  0.12478732,
          0.00699937, -0.00058141,  0.0937693 ,  0.04863384, -0.00416125,
         -0.06676556,  0.02907668, -0.02814799,  0.02993005, -0.00958559,
         -0.08623658,  0.00639441, -0.06963412, -0.01728812, -0.18978092,
         -0.02648345, -0.15898466,  0.

In [61]:
fake_recon_index[cid:cid + sl, :].T

memmap([[ 5.76127395e-02, -1.78871416e-02,  1.44217670e-01],
        [ 1.89544428e-02, -2.04251543e-01, -5.37966751e-02],
        [ 9.71924067e-02,  7.19601810e-02,  1.38117477e-01],
        [ 2.96681803e-02,  2.34301403e-01,  2.08711118e-01],
        [-7.05326274e-02,  4.17659506e-02,  6.50995374e-02],
        [ 3.80449332e-02,  9.19859335e-02,  9.03739259e-02],
        [ 2.53573246e-02, -9.89011154e-02,  2.17377767e-02],
        [ 8.30436498e-02,  7.30405599e-02,  1.29654929e-01],
        [-8.47351477e-02, -1.04567697e-02,  3.67555916e-02],
        [-6.12137653e-02,  6.16695732e-03, -3.67493518e-02],
        [ 1.22877248e-01,  2.86944676e-02,  9.05705243e-02],
        [-6.64657727e-02, -7.24025965e-02, -3.94301973e-02],
        [-8.96304622e-02,  8.10756348e-03,  3.63207906e-02],
        [ 9.18619707e-02, -3.82052474e-02, -1.09954908e-01],
        [ 9.10857022e-02,  1.00239366e-01,  3.04265227e-02],
        [-5.07972576e-02,  2.97043491e-02,  5.21308258e-02],
        [ 4.24566343e-02

In [62]:
fake_recon_index[cid:cid + sl, :].T.shape

(128, 3)

In [63]:
np.dot(q, fake_recon_index[cid:cid + sl, :].T) # fake_recon_index[3046219:3046220,:] #memmap([[0.6720861]], dtype=float32)

memmap([[0.67208624, 0.33104604, 0.35702977],
        [0.41469234, 0.28005293, 0.38899156],
        [0.4534766 , 0.21059127, 0.28456974]], dtype=float32)

In [64]:
np.diag(np.dot(q, fake_recon_index[cid:cid + sl, :].T))

memmap([0.67208624, 0.28005293, 0.28456974], dtype=float32)

In [65]:
np.mean(np.diag(np.dot(q, fake_recon_index[cid:cid + sl, :].T)))

0.4122363

In [294]:
_scores[ci] = np.mean(np.diag(np.dot(q, fake_recon_index[cid:cid + sl, :].T)))

In [295]:
_scores

array([0.22324252, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [296]:
np.argsort(-_scores) #ordena por ordem decrescente

array([ 0, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 66, 65, 64, 63, 62, 61, 35, 60, 58, 57, 56, 55, 54, 53, 59, 67,
       34, 32,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 31, 30, 29, 28, 27, 26, 33, 25, 23, 22, 21, 20, 19, 18, 24,
       68])

In [297]:
np.argsort(-_scores)[:10]  #ordena os valores por índices.

array([ 0, 36, 37, 38, 39, 40, 41, 42, 43, 44])

In [298]:
candidates[np.argsort(-_scores)[:10]]

array([   89, 10006, 10007, 10008, 10009, 10010, 10011, 10014, 10015,
       10016])

In [299]:
pred_ids = candidates[np.argsort(-_scores)[:10]]

In [300]:
gt_id

10024

In [301]:
pred_ids[0]

89

In [302]:
int(gt_id == pred_ids[0])

0

In [303]:
top1_exact[ti, si] = int(gt_id == pred_ids[0]) # (0, 0)

In [251]:
top1_exact

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [252]:
#top1_near
gt_id - 1 # 53768852
gt_id # 53768853
gt_id + 1 # 53768854

10025

In [253]:
pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1]

False

In [254]:
int(pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])

0

In [255]:
top1_near[ti, si] = int(pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])

In [256]:
top1_near

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [257]:
gt_id

10024

In [258]:
# top3
pred_ids[:3] #[ 3046219, 50021449, 46040053]

array([   89, 10006, 10007])

In [259]:
int(gt_id in pred_ids[:3])

0

In [260]:
top3_exact[ti, si] = int(gt_id in pred_ids[:3])

In [261]:
top3_exact

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [262]:
#top10 hit
pred_ids[:10] #[ 3046219, 50021449, 46040053, 44703573, 44703570, 44703566, 44703565, 39485449, 39399655, 32863495]

array([   89, 10006, 10007, 10008, 10009, 10010, 10011, 10014, 10015,
       10016])

In [263]:
int(gt_id in pred_ids[:10])

0

In [264]:
top10_exact[ti, si] = int(gt_id in pred_ids[:10])

In [265]:
top10_exact

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [ ]:
display_interval

5

In [ ]:
ti % display_interval

2

In [ ]:
((ti % display_interval) == 0)

False

In [ ]:
(ti != 0)

True

In [ ]:
top1_exact[:ti + 1, :],

(array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]),)

In [ ]:
np.mean(top1_exact[:ti + 1, :], axis=0)

array([0., 0., 0., 0., 0., 0.])

In [ ]:
100. * np.mean(top1_exact[:ti + 1, :], axis=0)

array([0., 0., 0., 0., 0., 0.])

In [ ]:
top1_exact_rate = 100. * np.mean(top1_exact[:ti + 1, :], axis=0)

In [ ]:
top1_near_rate = 100. * np.mean(top1_near[:ti + 1, :], axis=0)
top3_exact_rate = 100. * np.mean(top3_exact[:ti + 1, :], axis=0)
top10_exact_rate = 100. * np.mean(top10_exact[:ti + 1, :], axis=0)

In [71]:
gt=1
pre=1
print(int(gt == pre))

1


In [83]:
for ti, test_id in enumerate(test_ids):
    gt_id = gt_ids[ti]
    print(f"ti-{ti}-gt_id-{gt_id}-test_id-{test_id}")
    for si, sl in enumerate(test_seq_len):
        assert test_id <= len(query)
        q = query[test_id:(test_id + sl), :] # shape(q) = (length, dim)

        print(f"si-{si}-sl-{sl}")

        # segment-level top k search for each segment
        _, I = index.search(
            q, k_probe) # _: distance, I: result IDs matrix
        
        #print(f"ti-{ti}-test_id-{test_id}-si-{si}-sl-{sl}-I-{I}")

        # offset compensation to get the start IDs of candidate sequences
        for offset in range(len(I)):
            I[offset, :] -= offset
        
        print(f"I_offset-{I}")

        # unique candidates
        candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0
        #print(f"candidates-{candidates}")

        """ Sequence match score """
        _scores = np.zeros(len(candidates))
        for ci, cid in enumerate(candidates):
            _scores[ci] = np.mean(
                np.diag(
                    # np.dot(q, index.reconstruct_n(cid, (cid + l)).T)
                    np.dot(q, fake_recon_index[cid:cid + sl, :].T)
                    )
                )
        print(f"_scores-{_scores}")

        """ Evaluate """
        pred_ids = candidates[np.argsort(-_scores)[:10]]
        # pred_id = candidates[np.argmax(_scores)] <-- only top1-hit
        print(f"pred_ids-{pred_ids}")

        # top1 hit
        top1_exact[ti, si] = int(gt_id == pred_ids[0])
        top1_near[ti, si] = int(
            pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])
        # top1_song = need song info here...

        # top3, top10 hit
        top3_exact[ti, si] = int(gt_id in pred_ids[:3])
        top10_exact[ti, si] = int(gt_id in pred_ids[:10])

        if (ti != 0) & ((ti % display_interval) == 0):
            #avg_search_time = (time.time() - start_time) / display_interval \
            #    / len(test_seq_len)
            top1_exact_rate = 100. * np.mean(top1_exact[:ti + 1, :], axis=0)
            top1_near_rate = 100. * np.mean(top1_near[:ti + 1, :], axis=0)
            top3_exact_rate = 100. * np.mean(top3_exact[:ti + 1, :], axis=0)
            top10_exact_rate = 100. * np.mean(top10_exact[:ti + 1, :], axis=0)

print(f"top1_exact-{top1_exact}")
print(f"top1_near-{top1_near}")
print(f"top3_exact-{top3_exact}")
print(f"top10_exact-{top10_exact}")

print(f"\n\ntop1_exact_rate-{top1_exact_rate}")
print(f"top1_near_rate-{top1_near_rate}")
print(f"top3_exact-{top3_exact_rate}")
print(f"top10_exact_rate-{top10_exact_rate}")

ti-0-gt_id-53768853-test_id-14655
si-0-sl-1
I_offset-[[53768853 44703565 53768854 53768832 32602405  3046219 50090820 44703566
  32602390 30244778 25233445 50021449 53768875 28396311 46040053 27606284
  48439861 39399655 39485449 44703570]]
_scores-[0.67208612 0.65375686 0.64886057 0.64049262 0.66801822 0.66362274
 0.68814337 0.64285153 0.64139974 0.7005142  0.67302364 0.63727248
 0.64416903 0.64063817 0.65546489 0.65912843 0.68967021 0.77305609
 0.67796648 0.65430278]
pred_ids-[53768853 44703565 53768832 32602405 53768854 44703566  3046219 30244778
 32602390 50090820]
si-1-sl-3
I_offset-[[53768853 44703565 53768854 53768832 32602405  3046219 50090820 44703566
  32602390 30244778 25233445 50021449 53768875 28396311 46040053 27606284
  48439861 39399655 39485449 44703570]
 [53768854 53768875 50867758 23104133 53768853 53768855 53768876 50865420
  23104134 23104272 18441412 23104137 23104096 19577880 23104106 39696798
  23180609 19577879 23104138 50865249]
 [53768853 53768854 53768833 50

In [82]:
top1_exact_rate = 100. * np.mean(top1_exact, axis=0)
top1_near_rate = 100. * np.mean(top1_near, axis=0)
top3_exact_rate = 100. * np.mean(top3_exact, axis=0)
top10_exact_rate = 100. * np.mean(top10_exact, axis=0)

print(f"\n\ntop1_exact_rate-{top1_exact_rate}")
print(f"top1_near_rate-{top1_near_rate}")
print(f"top3_exact-{top3_exact_rate}")
print(f"top10_exact_rate-{top10_exact_rate}")



top1_exact_rate-[100. 100. 100.]
top1_near_rate-[100. 100. 100.]
top3_exact-[100. 100. 100.]
top10_exact_rate-[100. 100. 100.]


Antes de 25/06

In [ ]:
import os
import sys
import yaml
import glob
import h5py
import librosa

import numpy as np
import soundfile as sf
import tensorflow as tf

from model_RA.fp_RA.melspec.melspectrogram_RA import get_melspec_layer
from model_RA.fp_RA.nnfp import get_fingerprinter

2024-06-24 16:01:31.766660: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

config = "default_RA"
cfg = load_config(config)

cli: Configuration from ./config/default_RA.yaml


# Carregar chekcpoints

In [ ]:
def load_checkpoint(checkpoint_root_dir, checkpoint_name, checkpoint_index,
                    m_fp):
    """ Load a trained fingerprinter """
    # Create checkpoint
    checkpoint = tf.train.Checkpoint(model=m_fp)
    checkpoint_dir = checkpoint_root_dir + f'/{checkpoint_name}/'
    c_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir,
                                           max_to_keep=None)

    # Load
    if checkpoint_index == None:
        tf.print("\x1b[1;32mArgument 'checkpoint_index' was not specified.\x1b[0m")
        tf.print('\x1b[1;32mSearching for the latest checkpoint...\x1b[0m')
        latest_checkpoint = c_manager.latest_checkpoint
        if latest_checkpoint:
            checkpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])
            status = checkpoint.restore(latest_checkpoint)
            status.expect_partial()
            tf.print(f'---Restored from {c_manager.latest_checkpoint}---')
        else:
            raise FileNotFoundError(f'Cannot find checkpoint in {checkpoint_dir}')
    else:
        checkpoint_fpath = checkpoint_dir + 'ckpt-' + str(checkpoint_index)
        status = checkpoint.restore(checkpoint_fpath) # Let TF to handle error cases.
        status.expect_partial()
        tf.print(f'---Restored from {checkpoint_fpath}---')
    return checkpoint_index

# Carregar áudio

In [ ]:
source_root_dir = cfg['DIR']['SOURCE_ROOT_DIR']
audio_files = sorted(glob.glob(source_root_dir + 'test-dummy-db-100k-full/' + '**/*.wav', recursive=True))

In [ ]:
def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=8000)
    #audio, _ = sf.read(file_path)
    """
    audio, _ = sf.read(file_path)
    
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)  # Convertendo para mono se necessário
    # Normalizar o áudio
    audio = audio / np.max(np.abs(audio))
    """
    return audio

### Processar áudio

In [ ]:
def split_audio_into_segments(audio, dur=1.0, hop=.5, fs=8000):
    segment_samples = int(dur * fs) # 8000
    hop_samples = int(hop * fs) # 4000

    segments = []
    for start in range(0, len(audio) - segment_samples + 1, hop_samples):
        #len(audio) = 1898580; segment_samples = 8000; hop_samples = 4000
        #0, 1898580 - 8000 + 1, 4000

        #fica a faltar as residual frames, que são as ultimas que são menores que 1 segundo

        segment = audio[start:start + segment_samples] #faz os vetores de 8000 em 8000, isto é, de 1s em 1s, com 

        if len(segment) == segment_samples:
            segments.append(segment)

    return segments

In [ ]:
dur=cfg['MODEL']['DUR']
hop=cfg['MODEL']['HOP']
fs=cfg['MODEL']['FS']

Embedding

In [ ]:
def build_fp(cfg):
    """ Build fingerprinter """
    # m_pre: log-power-Mel-spectrogram layer, S.
    m_pre = get_melspec_layer(cfg, trainable=False)

    # m_fp: fingerprinter g(f(.)).
    m_fp = get_fingerprinter(cfg, trainable=False)
    return m_pre, m_fp

@tf.function
def embeddingGenerator(X, m_pre, m_fp):
    """ 
    X -> (B,1,8000)
    """
    #tf.print(f"X:{X}")
    feat = m_pre(X)  # (nA+nP, F, T, 1)
    m_fp.trainable = False
    emb_f = m_fp.front_conv(feat)  # (BSZ, Dim)
    emb_gf = m_fp.div_enc(emb_f)
    emb_gf = tf.math.l2_normalize(emb_gf, axis=1)
    return emb_gf # f(.), L2(f(.)), L2(g(f(.))

In [ ]:
m_pre, m_fp = build_fp(cfg)
checkpoint_root_dir:str = "./logs/CHECK_BFTRI_100/"

checkpoint = tf.train.Checkpoint(m_fp)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_root_dir))

output_root_dir = '/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummy_db/'

In [ ]:
def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
for file_path in audio_files[2:3]:
    base_name = os.path.splitext(os.path.basename(file_path))[0]

    dir_name = f'{base_name[:3]}'
    print(dir_name)
    print(file_path)

    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    audio = load_audio(file_path)
    segments = split_audio_into_segments(audio, dur, hop, fs)
    X_segments = tf.convert_to_tensor(segments, dtype=tf.float32)

    #Tendo cada segmento do audio na forma de tensor de tamanho 1s a um hop de 0.5s, pode-se converter cada segmento num embedding
    emb = [] 
    for i in range(len(X_segments)):
        X = tf.reshape(X_segments[i],(1, 1,-1)) # podia meter um append, mas pode ser uma variável auxiliar, porque só preciso do tensor para gerar o embedding

        embedding = embeddingGenerator(X, m_pre, m_fp)
        emb.append(embedding.numpy())
    
    #Tendo todos os segementos do audio em vetores embedding, pode-se criar um ficheiro .h5 para o audio,pode-se fazer assim:
    #000003.h5
    #vetores:[vetor1, vetor2, vetor3], sendo total de vetores igual ao número de segmentos 
    emb_array = np.array(emb)

    output_file_path = os.path.join(output_root_dir +  dir_name + '/', base_name + '.h5')
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with h5py.File(output_file_path, 'w') as hf:
            a=1#hf.create_dataset('embeddings', data=emb_array)

000
/mnt/dataset/public/Fingerprinting/neural-audio-fp-dataset/music/test-dummy-db-100k-full/fma_full/000/000026.wav


KeyboardInterrupt: 

In [ ]:
import h5py
import numpy as np

def visualizar_dados_hdf5(file_path):
    emb_array = [] 
    with h5py.File(file_path, 'r') as hf:
        # Verifica se o dataset 'embeddings' está presente
        if 'embeddings' in hf:
            # Lê os dados do dataset 'embeddings'
            emb_array.append(hf['embeddings'][:])
            print(f'Dados do arquivo {file_path}:')
        else:
            print(f'Dataset "embeddings" não encontrado no arquivo {file_path}')
    return emb_array


h5_file = '/mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummy_db/000003.h5'

x=visualizar_dados_hdf5(h5_file)

Dados do arquivo /mnt/dataset/public/Fingerprinting/Embeddings_BFTRI/dummy_db/000003.h5:


In [ ]:
# -*- coding: utf-8 -*-
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.
""" eval_faiss.py """
import os
import sys
import time
import glob
import click
import curses
import pathlib
import yaml
import numpy as np
#sys.path.append(os.path.abspath('/eval_RA/'))
#sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(__file__))))
from utils.get_index_faiss import get_index
from utils.print_table import PrintTable
import faiss

In [ ]:
def load_config(config_fname):
    config_filepath = '../config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [ ]:
config = "default_RA"
cfg = load_config(config)

cli: Configuration from ./config/default_RA.yaml


In [ ]:
def load_memmap_data(source_dir,
                     fname,
                     append_extra_length=None,
                     shape_only=False,
                     display=True):

    path_shape = source_dir + fname + '_shape.npy'
    path_data = source_dir + fname + '.mm'
    data_shape = np.load(path_shape)
    if shape_only:
        return data_shape

    if append_extra_length:
        data_shape[0] += append_extra_length
        data = np.memmap(path_data, dtype='float32', mode='r+',
                         shape=(data_shape[0], data_shape[1]))
    else:
        data = np.memmap(path_data, dtype='float32', mode='r',
                         shape=(data_shape[0], data_shape[1]))
    if display:
        print(f'Load {data_shape[0]:,} items from \033[32m{path_data}\033[0m.')
    return data, data_shape

In [ ]:
logsDir = '/mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101'
emb_dir = logsDir + '/'

emb_dummy_dir = None
index_type='ivfpq'
nogpu=False
max_train=1e7
test_ids='icassp'
test_seq_len='1 3 5 9 11 19'
k_probe=20
display_interval=5

In [ ]:
#test_seq_len='1'

test_seq_len = np.asarray(
        list(map(int, test_seq_len.split())))  # '1 3 5' --> [1, 3, 5]

query, query_shape = load_memmap_data(emb_dir, 'query')
db, db_shape = load_memmap_data(emb_dir, 'db')

if emb_dummy_dir is None:
    emb_dummy_dir = emb_dir

dummy_db, dummy_db_shape = load_memmap_data(emb_dummy_dir, 'dummy_db')

Load 29,500 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/query.mm.
Load 29,500 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/db.mm.
Load 53,754,198 items from /mnt/dev/rodrigoalmeida/neural-audio-fp/logs/emb/CHECK_BFTRI_100/101/dummy_db.mm.


In [ ]:
print(dummy_db.shape, dummy_db_shape, db.shape, db_shape, query.shape, query_shape)

(53754198, 128) [53754198      128] (29500, 128) [29500   128] (29500, 128) [29500   128]


In [ ]:
def create_index(dummy_db, dummy_db_shape):
    #d = dummy_db_shape[1]

    # Build a flat (CPU) index
    index = faiss.IndexFlatL2(128) #

    index_type = 'ivfpq'
    mode = index_type.lower()
    print(f'Creating index: \033[93m{mode}\033[0m')

    # Using IVF-PQ index
    code_sz = 64 # power of 2
    n_centroids = 256#
    nbits = 8  # nbits must be 8, 12 or 16, The dimension d should be a multiple of M.
    index = faiss.IndexIVFPQ(index, 128, n_centroids, code_sz, nbits)

    train_data = dummy_db
    max_train=1e7
    max_nitem_train = int(max_train)

    # Train index

    if len(train_data) > max_nitem_train:
        print('Training index using {:>3.2f} % of data...'.format(
            100. * max_nitem_train / len(train_data)))
        # shuffle and reduce training data
        sel_tr_idx = np.random.permutation(len(train_data))
        sel_tr_idx = sel_tr_idx[:max_nitem_train]
        index.train(train_data[sel_tr_idx,:])

    index.nprobe = 40
    print(f"index trained: {index.is_trained}")

    #index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')
    #index.add(db); print(f'{len(db)} items from reference DB') #corresponde aos que estão verdadeiros e iguais ao query_db
    return index

In [ ]:
index = create_index(dummy_db, dummy_db_shape) # Demorou 22m36.2s sem o index.add

Creating index: ivfpq
Training index using 18.60 % of data...
index trained: True


In [ ]:
index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')
index.add(db); print(f'{len(db)} items from reference DB') #corresponde aos que estão verdadeiros e iguais ao query_db

53754198 items from dummy DB
29500 items from reference DB


In [ ]:
# Get test_ids
print(f'test_id: \033[93m{test_ids}\033[0m,  ', end='') #icassp

test_id: [14655 21662  7594 ... 15722  1097  4538],  

In [ ]:
if test_ids.lower() == 'all':
    print('1')
    test_ids = np.arange(0, len(query) - max(test_seq_len), 1) # will test all segments in query/db set
elif test_ids.lower() == 'icassp':
    print('2')
    test_ids = np.load(
        glob.glob('../**/test_ids_icassp2021.npy', recursive=True)[0])
elif test_ids.isnumeric():
    print('3')
    test_ids = np.random.permutation(len(query) - max(test_seq_len))[:int(test_ids)]
else:
    print('4')
    test_ids = np.load(test_ids)

n_test = len(test_ids)
gt_ids  = test_ids + dummy_db_shape[0]
print(f'n_test: \033[93m{n_test:n}\033[0m')

2
n_test: 2000


In [ ]:
test_ids[0:250]

array([14655, 21662,  7594, 23837,  6986,   503,   112, 25776, 11851,
       15398, 22953, 13264,  6108, 16048, 22217, 13886, 12292,  7189,
        1904, 20861, 17157, 23825, 18234, 21862, 29201, 17855, 27441,
       14694,  3954,   483,  8438, 27553,  1310, 22681, 21222,  6360,
        3637,  2577, 27481, 27701, 13465,  7457,  3170,  1357, 25567,
       20961, 23649, 11466, 14353,  9081, 26635, 11388, 20681,  8941,
        2021, 12345, 10838, 27587, 21255, 11784,  3838,  8001, 11629,
       15450,  4809,  1790, 22565,  8527, 26434,  9016, 20142,  3558,
        2094, 16883, 24742, 18908, 15654, 27950, 18123, 27771,  1275,
       16603, 10512, 20703, 28558, 15078, 13893,  2152,  7797, 27343,
       26346,  9864, 27904, 27122, 28266, 12375, 23038, 16580, 27227,
        2646,  5623, 17025, 21296, 27146, 20650, 14627, 11486, 23879,
       25857, 10332,  4860, 25424,  6348,  3334, 28383, 24786,  6867,
       25243,  9026,  7575, 17355, 21779,  1507,  6790, 24307,  9351,
       19457,  6386,

o test_ids contém 2000 indices (?), isto é, dado uma música


ver quantas musicas tem o dummy_db,
ver quantos vetores tem cada musica e qual a ordem, para saber que indice é cada vetor e a que musica pertence

In [ ]:
"""
import numpy as np
import pandas as pd

# Exemplo de array que você quer salvar
data = test_ids

# Criar um DataFrame com os dados
df = pd.DataFrame(data, columns=['indice'])

# Escolher o nome do arquivo CSV
nome_arquivo = 'test_ids.csv'

# Salvar o DataFrame em um arquivo CSV
df.to_csv(nome_arquivo, index=False)

print(f'Arquivo {nome_arquivo} salvo com sucesso.')
"""


In [ ]:
""" Segement/sequence-level search & evaluation """
# Define metric
top1_exact = np.zeros((n_test, len(test_seq_len))).astype(int) # (n_test, test_seg_len)
top1_near = np.zeros((n_test, len(test_seq_len))).astype(int)
# top1_song = np.zeros((n_test, len(test_seq_len))).astype(np.int)

In [ ]:
query[ 14655 : 14655 + 1 , :] #14655 é o primeiro valor do test_ids, sl é o sequence_length value.
#ou seja, do query, dos 60 vetores, escolhe-se o indice 14655, quer-se avançar 1 vetor (1 segundo), logo vai de 14655 a 14656.
#A comparação vai ser feita no query[14655:14656, :]
#q = query[test_id:(test_id + sl), :]

memmap([[ 0.15809532, -0.02599731,  0.01563343, -0.01498089, -0.08509405,
          0.00918309,  0.07498254,  0.14784455,  0.04171429,  0.00654791,
          0.15193549,  0.0434388 , -0.08942915,  0.20740546,  0.03593424,
         -0.06072055, -0.04684889, -0.09035619, -0.06467165, -0.00962286,
         -0.01251085, -0.08126785,  0.00619965, -0.04982934, -0.00722066,
         -0.0855777 ,  0.17162009, -0.07458217,  0.04839539,  0.00616283,
          0.02169314, -0.07166849,  0.14735067,  0.02083374,  0.1120945 ,
          0.07894135, -0.0452273 , -0.10079931, -0.08433416, -0.0737057 ,
         -0.12632951, -0.10347243, -0.06913505,  0.07233108, -0.09519916,
          0.08077845, -0.03659033,  0.01063322, -0.11243182,  0.12478732,
          0.00699937, -0.00058141,  0.0937693 ,  0.04863384, -0.00416125,
         -0.06676556,  0.02907668, -0.02814799,  0.02993005, -0.00958559,
         -0.08623658,  0.00639441, -0.06963412, -0.01728812, -0.18978092,
         -0.02648345, -0.15898466,  0.

In [ ]:
gt_ids #test_ids + dummy_db_shape[0]

array([53768853, 53775860, 53761792, ..., 53769920, 53755295, 53758736])

In [ ]:
test_id = test_ids[0]

In [ ]:
sl = test_seq_len[1]

In [ ]:
q = query[test_id:(test_id + sl), :]

In [ ]:
_, I = index.search(q, k_probe)

In [ ]:
print(len(I))

3


In [ ]:
II=I

In [ ]:
II

array([[53768853, 44703565, 53768832, 32602405, 44703566, 53768854,
         3046219, 30244778, 50021449, 32602390, 50090820, 46040053,
        53768875,  5261943, 39399655, 28396311, 25233445, 46040054,
        44703570, 27606284],
       [53768855, 53768876, 50867759, 23104134, 53768854, 50865421,
        23104135, 53768877, 53768856, 23104097, 19577881, 23104138,
        18441413, 23104273, 23104136, 23104107, 19577880, 23104010,
        23180610, 38286572],
       [53768855, 53768835, 53768856, 30777645, 53768834, 32228339,
        53768876, 32216725, 50865430, 32228337, 53768878, 35513917,
        30803668, 50865436, 30803754, 21509771, 51635979, 23180587,
        53768877, 50865313]])

In [ ]:
II[2, :] -= 2

In [ ]:
II

array([[53768853, 44703565, 53768832, 32602405, 44703566, 53768854,
         3046219, 30244778, 50021449, 32602390, 50090820, 46040053,
        53768875,  5261943, 39399655, 28396311, 25233445, 46040054,
        44703570, 27606284],
       [53768854, 53768875, 50867758, 23104133, 53768853, 50865420,
        23104134, 53768876, 53768855, 23104096, 19577880, 23104137,
        18441412, 23104272, 23104135, 23104106, 19577879, 23104009,
        23180609, 38286571],
       [53768853, 53768833, 53768854, 30777643, 53768832, 32228337,
        53768874, 32216723, 50865428, 32228335, 53768876, 35513915,
        30803666, 50865434, 30803752, 21509769, 51635977, 23180585,
        53768875, 50865311]])

In [ ]:
for offset in range(len(I)):
    I[offset, :] -= offset

In [ ]:
I

array([[53768853, 44703565, 53768832, 32602405, 44703566, 53768854,
         3046219, 30244778, 50021449, 32602390, 50090820, 46040053,
        53768875,  5261943, 39399655, 28396311, 25233445, 46040054,
        44703570, 27606284],
       [53768853, 53768874, 50867757, 23104132, 53768852, 50865419,
        23104133, 53768875, 53768854, 23104095, 19577879, 23104136,
        18441411, 23104271, 23104134, 23104105, 19577878, 23104008,
        23180608, 38286570],
       [53768851, 53768831, 53768852, 30777641, 53768830, 32228335,
        53768872, 32216721, 50865426, 32228333, 53768874, 35513913,
        30803664, 50865432, 30803750, 21509767, 51635975, 23180583,
        53768873, 50865309]])

In [ ]:
candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0
print(candidates)

[ 3046219  5261943 18441411 19577878 19577879 21509767 23104008 23104095
 23104105 23104132 23104133 23104134 23104136 23104271 23180583 23180608
 25233445 27606284 28396311 30244778 30777641 30803664 30803750 32216721
 32228333 32228335 32602390 32602405 35513913 38286570 39399655 44703565
 44703566 44703570 46040053 46040054 50021449 50090820 50865309 50865419
 50865426 50865432 50867757 51635975 53768830 53768831 53768832 53768851
 53768852 53768853 53768854 53768872 53768873 53768874 53768875]


In [ ]:
""" Sequence match score """
_scores = np.zeros(len(candidates))

In [ ]:
for ci, cid in enumerate(candidates):
    print(ci, cid)

0 3046219
1 5261943
2 18441411
3 19577878
4 19577879
5 21509767
6 23104008
7 23104095
8 23104105
9 23104132
10 23104133
11 23104134
12 23104136
13 23104271
14 23180583
15 23180608
16 25233445
17 27606284
18 28396311
19 30244778
20 30777641
21 30803664
22 30803750
23 32216721
24 32228333
25 32228335
26 32602390
27 32602405
28 35513913
29 38286570
30 39399655
31 44703565
32 44703566
33 44703570
34 46040053
35 46040054
36 50021449
37 50090820
38 50865309
39 50865419
40 50865426
41 50865432
42 50867757
43 51635975
44 53768830
45 53768831
46 53768832
47 53768851
48 53768852
49 53768853
50 53768854
51 53768872
52 53768873
53 53768874
54 53768875


In [ ]:
emb_dummy_dir = emb_dir

In [ ]:
del dummy_db

fake_recon_index, index_shape = load_memmap_data(
    emb_dummy_dir, 'dummy_db', append_extra_length=query_shape[0],
    display=False)
#len(fake_recon_index) - 53783698

In [ ]:
fake_recon_index[dummy_db_shape[0]:dummy_db_shape[0] + query_shape[0], :] = db[:, :]
fake_recon_index.flush()
#len(fake_recon_index) - 53783698

In [ ]:
ci = 0
cid = 153702

In [ ]:
np.dot(q, fake_recon_index[cid:cid + sl, :].T)

memmap([[ 0.13540626,  0.10583296,  0.00773416,  0.01316234, -0.02899436,
         -0.04654417, -0.00783276, -0.11456508, -0.04313429, -0.03091665,
         -0.0466799 ],
        [ 0.1649779 ,  0.1344586 ,  0.03165737,  0.07611004,  0.07867202,
         -0.04457755,  0.07102919,  0.13729277,  0.08224892,  0.05492847,
          0.12838879],
        [ 0.06412753,  0.02963007, -0.01620481,  0.04184759,  0.06647854,
          0.01239931,  0.05275577,  0.119441  ,  0.06510368,  0.11412318,
          0.08172489],
        [ 0.10762019, -0.00687964,  0.07002341,  0.15537184,  0.12973554,
          0.01866154,  0.09201331,  0.0503825 ,  0.15598398,  0.1723681 ,
          0.08008766],
        [ 0.05130191, -0.02845777,  0.09642363,  0.07476643,  0.11307297,
         -0.00240691,  0.0151294 ,  0.0215014 ,  0.03398096,  0.10673533,
          0.08688808],
        [ 0.08944897,  0.0128467 ,  0.0403959 ,  0.16369176,  0.18174008,
          0.06037472,  0.0693492 , -0.00599509,  0.11195889,  0.1619095

In [ ]:
np.diag(np.dot(q, fake_recon_index[cid:cid + sl, :].T))

memmap([ 0.13540626,  0.1344586 , -0.01620481,  0.15537184,  0.11307297,
         0.06037472,  0.15101886,  0.17358014,  0.06591691,  0.11153555,
         0.09477568], dtype=float32)

In [ ]:
np.mean(
        np.diag(
            # np.dot(q, index.reconstruct_n(cid, (cid + l)).T)
            np.dot(q, fake_recon_index[cid:cid + sl, :].T)
            )
        )

0.107209705

In [ ]:
np.argsort(-_scores)[:10]

array([  0, 111, 112, 113, 114, 115, 116, 117, 118, 110])

In [ ]:
candidates[np.argsort(-_scores)[:10]]

array([  153702, 39665315, 39665320, 39665424, 39665429, 39665435,
       39665473, 39753136, 40748170, 39497959])

In [ ]:
pred_ids = candidates[np.argsort(-_scores)[:10]]

In [ ]:
ti=0
si=0
gt_id = gt_ids[ti]

In [ ]:
gt_id

53768853

In [ ]:
int(gt_id == pred_ids[0])

0

In [ ]:
int(
    pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])

0

In [ ]:
top1_exact[ti, si] = int(gt_id == pred_ids[0])
top1_near[ti, si] = int(
    pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])

In [ ]:
for ti, test_id in enumerate(test_ids):
        gt_id = gt_ids[ti]
        print(ti)
        for si, sl in enumerate(test_seq_len):
            assert test_id <= len(query)
            q = query[test_id:(test_id + sl), :] # shape(q) = (length, dim)

            # segment-level top k search for each segment
            _, I = index.search(
                q, k_probe) # _: distance, I: result IDs matrix

            # offset compensation to get the start IDs of candidate sequences
            for offset in range(len(I)):
                I[offset, :] -= offset

            # unique candidates
            candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0

            """ Sequence match score """
            _scores = np.zeros(len(candidates))
            for ci, cid in enumerate(candidates):
                _scores[ci] = np.mean(
                    np.diag(
                        # np.dot(q, index.reconstruct_n(cid, (cid + l)).T)
                        np.dot(q, fake_recon_index[cid:cid + sl, :].T)
                        )
                    )

            """ Evaluate """
            pred_ids = candidates[np.argsort(-_scores)[:10]]
            # pred_id = candidates[np.argmax(_scores)] <-- only top1-hit

            # top1 hit
            top1_exact[ti, si] = int(gt_id == pred_ids[0])
            top1_near[ti, si] = int(
                pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])
            # top1_song = need song info here...


        if (ti != 0) & ((ti % display_interval) == 0):
            top1_exact_rate = 100. * np.mean(top1_exact[:ti + 1, :], axis=0)
            top1_near_rate = 100. * np.mean(top1_near[:ti + 1, :], axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# Summary
top1_exact_rate = 100. * np.mean(top1_exact, axis=0)
top1_near_rate = 100. * np.mean(top1_near, axis=0)

In [ ]:
top1_exact_rate

array([70.6 , 91.65, 96.  , 98.05, 98.75, 99.35])

In [ ]:
top1_near_rate

array([74.25, 92.15, 96.05, 98.1 , 98.75, 99.35])

In [ ]:
for ci, cid in enumerate(candidates):
    _scores[ci] = np.mean(
        np.diag(
            # np.dot(q, index.reconstruct_n(cid, (cid + l)).T)
            np.dot(q, fake_recon_index[cid:cid + sl, :].T)
            )
        )

In [ ]:
for ti, test_id in enumerate(test_ids):
    gt_id = gt_ids[ti]
    #print(test_id)
    for si, sl in enumerate(test_seq_len):
        #print(sl)
        q = query[test_id:(test_id + sl), :] # query[ 14655 : 14655 + 1 , :]

        # segment-level top k search for each segment
        _, I = index.search(q, k_probe) # _: distance, I: result IDs matrix

        print(len(I))

        for offset in range(len(I)):
            I[offset, :] -= offset

        candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0
        print(candidates)

In [ ]:

    
       
            assert test_id <= len(query)
            q = query[test_id:(test_id + sl), :] # shape(q) = (length, dim)

            # segment-level top k search for each segment
            _, I = index.search(
                q, k_probe) # _: distance, I: result IDs matrix

            # offset compensation to get the start IDs of candidate sequences
            for offset in range(len(I)):
                I[offset, :] -= offset

            # unique candidates
            candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0

            """ Sequence match score """
            _scores = np.zeros(len(candidates))
            for ci, cid in enumerate(candidates):
                _scores[ci] = np.mean(
                    np.diag(
                        # np.dot(q, index.reconstruct_n(cid, (cid + l)).T)
                        np.dot(q, fake_recon_index[cid:cid + sl, :].T)
                        )
                    )

            """ Evaluate """
            pred_ids = candidates[np.argsort(-_scores)[:10]]
            # pred_id = candidates[np.argmax(_scores)] <-- only top1-hit

            # top1 hit
            top1_exact[ti, si] = int(gt_id == pred_ids[0])
            top1_near[ti, si] = int(
                pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])
            # top1_song = need song info here...

            # top3, top10 hit
            top3_exact[ti, si] = int(gt_id in pred_ids[:3])
            top10_exact[ti, si] = int(gt_id in pred_ids[:10])


        if (ti != 0) & ((ti % display_interval) == 0):
            avg_search_time = (time.time() - start_time) / display_interval \
                / len(test_seq_len)
            top1_exact_rate = 100. * np.mean(top1_exact[:ti + 1, :], axis=0)
            top1_near_rate = 100. * np.mean(top1_near[:ti + 1, :], axis=0)
            top3_exact_rate = 100. * np.mean(top3_exact[:ti + 1, :], axis=0)
            top10_exact_rate = 100. * np.mean(top10_exact[:ti + 1, :], axis=0)
            # top1_song = 100 * np.mean(tp_song[:ti + 1, :], axis=0)

            pt.update_counter(ti, n_test, avg_search_time * 1000.)
            pt.update_table((top1_exact_rate, top1_near_rate, top3_exact_rate,
                             top10_exact_rate))
            start_time = time.time() # reset stopwatch

In [ ]:
def eval_faiss(emb_dir,
               emb_dummy_dir=None,
               index_type='ivfpq',
               nogpu=False,
               max_train=1e7,
               test_ids='icassp',
               test_seq_len='1 3 5 9 11 19',
               k_probe=20,
               display_interval=5):
    """
    Segment/sequence-wise audio search experiment and evaluation: implementation based on FAISS.

    ex) python eval.py EMB_DIR --index_type ivfpq

    EMB_DIR: Directory where {query, db, dummy_db}.mm files are located. The 'raw_score.npy' and 'test_ids.npy' will be also created in the same directory.



    """
    test_seq_len = np.asarray(
        list(map(int, test_seq_len.split())))  # '1 3 5' --> [1, 3, 5]

    # Load items from {query, db, dummy_db}
    query, query_shape = load_memmap_data(emb_dir, 'query')
    db, db_shape = load_memmap_data(emb_dir, 'db')
    if emb_dummy_dir is None:
        emb_dummy_dir = emb_dir
    dummy_db, dummy_db_shape = load_memmap_data(emb_dummy_dir, 'dummy_db')
    """ ----------------------------------------------------------------------
    FAISS index setup

        dummy: 10 items.
        db: 5 items.
        query: 5 items, corresponding to 'db'.

        index.add(dummy_db); index.add(db) # 'dummy_db' first

               |------ dummy_db ------|
        index: [d0, d1, d2,..., d8, d9, d11, d12, d13, d14, d15]
                                       |--------- db ----------|

                                       |--------query ---------|
                                       [q0,  q1,  q2,  q3,  q4]

    • The set of ground truth IDs for q[i] will be (i + len(dummy_db))

    ---------------------------------------------------------------------- """
    # Create and train FAISS index
    index = get_index(index_type, dummy_db, dummy_db.shape, (not nogpu),
                      max_train)

    # Add items to index
    start_time = time.time()

    index.add(dummy_db); print(f'{len(dummy_db)} items from dummy DB')
    index.add(db); print(f'{len(db)} items from reference DB')

    t = time.time() - start_time
    print(f'Added total {index.ntotal} items to DB. {t:>4.2f} sec.')

    """ ----------------------------------------------------------------------
    We need to prepare a merged {dummy_db + db} memmap:

    • Calcuation of sequence-level matching score requires reconstruction of
      vectors from FAISS index.
    • Unforunately, current faiss.index.reconstruct_n(id_start, id_stop)
      supports only CPU index.
    • We prepare a fake_recon_index thourgh the on-disk method.

    ---------------------------------------------------------------------- """
    # Prepare fake_recon_index
    del dummy_db
    start_time = time.time()

    fake_recon_index, index_shape = load_memmap_data(
        emb_dummy_dir, 'dummy_db', append_extra_length=query_shape[0],
        display=False)
    fake_recon_index[dummy_db_shape[0]:dummy_db_shape[0] + query_shape[0], :] = db[:, :]
    fake_recon_index.flush()

    t = time.time() - start_time
    print(f'Created fake_recon_index, total {index_shape[0]} items. {t:>4.2f} sec.')

    # Get test_ids
    print(f'test_id: \033[93m{test_ids}\033[0m,  ', end='')
    if test_ids.lower() == 'all':
        test_ids = np.arange(0, len(query) - max(test_seq_len), 1) # will test all segments in query/db set
    elif test_ids.lower() == 'icassp':
        test_ids = np.load(
            glob.glob('./**/test_ids_icassp2021.npy', recursive=True)[0])
    elif test_ids.isnumeric():
        test_ids = np.random.permutation(len(query) - max(test_seq_len))[:int(test_ids)]
    else:
        test_ids = np.load(test_ids)

    n_test = len(test_ids)
    gt_ids  = test_ids + dummy_db_shape[0]
    print(f'n_test: \033[93m{n_test:n}\033[0m')

    """ Segement/sequence-level search & evaluation """
    # Define metric
    top1_exact = np.zeros((n_test, len(test_seq_len))).astype(int) # (n_test, test_seg_len)
    top1_near = np.zeros((n_test, len(test_seq_len))).astype(int)
    top3_exact = np.zeros((n_test, len(test_seq_len))).astype(int)
    top10_exact = np.zeros((n_test, len(test_seq_len))).astype(int)
    # top1_song = np.zeros((n_test, len(test_seq_len))).astype(np.int)

    scr = curses.initscr()
    pt = PrintTable(scr=scr, test_seq_len=test_seq_len,
                    row_names=['Top1 exact', 'Top1 near', 'Top3 exact','Top10 exact'])
    start_time = time.time()
    for ti, test_id in enumerate(test_ids):
        gt_id = gt_ids[ti]
        for si, sl in enumerate(test_seq_len):
            assert test_id <= len(query)
            q = query[test_id:(test_id + sl), :] # shape(q) = (length, dim)

            # segment-level top k search for each segment
            _, I = index.search(
                q, k_probe) # _: distance, I: result IDs matrix

            # offset compensation to get the start IDs of candidate sequences
            for offset in range(len(I)):
                I[offset, :] -= offset

            # unique candidates
            candidates = np.unique(I[np.where(I >= 0)])   # ignore id < 0

            """ Sequence match score """
            _scores = np.zeros(len(candidates))
            for ci, cid in enumerate(candidates):
                _scores[ci] = np.mean(
                    np.diag(
                        # np.dot(q, index.reconstruct_n(cid, (cid + l)).T)
                        np.dot(q, fake_recon_index[cid:cid + sl, :].T)
                        )
                    )

            """ Evaluate """
            pred_ids = candidates[np.argsort(-_scores)[:10]]
            # pred_id = candidates[np.argmax(_scores)] <-- only top1-hit

            # top1 hit
            top1_exact[ti, si] = int(gt_id == pred_ids[0])
            top1_near[ti, si] = int(
                pred_ids[0] in [gt_id - 1, gt_id, gt_id + 1])
            # top1_song = need song info here...

            # top3, top10 hit
            top3_exact[ti, si] = int(gt_id in pred_ids[:3])
            top10_exact[ti, si] = int(gt_id in pred_ids[:10])


        if (ti != 0) & ((ti % display_interval) == 0):
            avg_search_time = (time.time() - start_time) / display_interval \
                / len(test_seq_len)
            top1_exact_rate = 100. * np.mean(top1_exact[:ti + 1, :], axis=0)
            top1_near_rate = 100. * np.mean(top1_near[:ti + 1, :], axis=0)
            top3_exact_rate = 100. * np.mean(top3_exact[:ti + 1, :], axis=0)
            top10_exact_rate = 100. * np.mean(top10_exact[:ti + 1, :], axis=0)
            # top1_song = 100 * np.mean(tp_song[:ti + 1, :], axis=0)

            pt.update_counter(ti, n_test, avg_search_time * 1000.)
            pt.update_table((top1_exact_rate, top1_near_rate, top3_exact_rate,
                             top10_exact_rate))
            start_time = time.time() # reset stopwatch

    # Summary
    top1_exact_rate = 100. * np.mean(top1_exact, axis=0)
    top1_near_rate = 100. * np.mean(top1_near, axis=0)
    top3_exact_rate = 100. * np.mean(top3_exact, axis=0)
    top10_exact_rate = 100. * np.mean(top10_exact, axis=0)
    # top1_song = 100 * np.mean(top1_song[:ti + 1, :], axis=0)

    pt.update_counter(ti, n_test, avg_search_time * 1000.)
    pt.update_table((top1_exact_rate, top1_near_rate, top3_exact_rate, top10_exact_rate))
    pt.close_table() # close table and print summary
    del fake_recon_index, query, db
    np.save(f'{emb_dir}/raw_score.npy',
            np.concatenate(
                (top1_exact, top1_near, top3_exact, top10_exact), axis=1))
    np.save(f'{emb_dir}/test_ids.npy', test_ids)
    print(f'Saved test_ids and raw score to {emb_dir}.')

if __name__ == "__main__":
    curses.wrapper(eval_faiss())

In [ ]:
eval_faiss(emb_dir,
            emb_dummy_dir,
            index_type,
            nogpu,
            max_train,
            test_ids,
            test_seq_len,
            k_probe,
            display_interval)